In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.mllib.recommendation import *
conf = SparkConf().setAppName("ALS")
sc = SparkContext()

In [2]:
def parse_artist(x):
    try:
        return int(x)
    except:
        pass

In [7]:
rawArtistData = sc.textFile("artist_data.txt")

In [8]:
artistByID = rawArtistData.map(lambda x: x.split('\t'))\
                        .filter(lambda x: len(x) == 2)\
                          .map(lambda x: [parse_artist(x[0]), x[1]])

In [9]:
def parse_alias(x,y):
    try:
        return [int(x), int(y)]
    except:
        pass

In [10]:
rawArtistAlias = sc.textFile("artist_alias.txt")

In [11]:
artistAlias = rawArtistAlias.map(lambda x: x.split('\t'))\
                             .filter(lambda x : x[0] != '')\
                            .map(lambda x: parse_alias(x[0], x[1]))\
                            .collectAsMap()

In [12]:
def convert_to_goodId(x):
    if x[1] in artistAlias.keys():
        x[1] = artistAlias.get(x[1])
    return [x[0], x[1], x[2]]

In [13]:
rawUserArtistData = sc.textFile("user_artist_data.txt")

In [14]:
UserArtistData = rawUserArtistData.map(lambda x: x.split(' '))\
                                  .map(lambda x: [int(x[0]), int(x[1]), int(x[2])])\
                                    .map(convert_to_goodId)\
                                .cache()

In [15]:
model = ALS.trainImplicit(UserArtistData, 10, 5, 0.01)

In [18]:
recommendation = model.recommendProducts (2093760 , 10)

In [25]:
artistMap=artistByID.collectAsMap()
for i in recommendation:
    print(artistMap.get(i.product), i.rating)

None 0.9975880420728608
None 0.997503918719121
None 0.9960366682056853
None 0.9958308696239973
None 0.9954875199246301
None 0.9948097969351956
None 0.9940974477627781
None 0.9940290297217184
None 0.9935819594191873
None 0.9935416299294184
